*Big Data Final Project *

Stav Vaknin -313581654

Guy Assa - 204118616

In [1]:
RUNNING_ENV = 'CLOUD'
#RUNNING_ENV = 'LOCAL'

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1629388517974_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
if RUNNING_ENV is 'CLOUD':
    sc.install_pypi_package("Cython")
    sc.install_pypi_package("pandas==0.25.1")
    sc.install_pypi_package("kaggle")
    sc.install_pypi_package("matplotlib")
    sc.install_pypi_package("scikit-learn")
    sc.install_pypi_package("plotly")
else:
    pass
    #!pip install pyspark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/3d/48/bbca549da0b0f636c0f161e84d30172c40aafe99552680f297da7fedf102/Cython-0.29.24-cp37-cp37m-manylinux1_x86_64.whl

  Using cached https://files.pythonhosted.org/packages/7e/ab/ea76361f9d3e732e114adcd801d2820d5319c23d0ac5482fa3b412db217e/pandas-0.25.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/05/1b/0a0dece0e8aa492a6ec9e4ad2fe366b511558cdc73fd3abc82ba7348e875/certifi-2021.5.30-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/92/96/144f70b972a9c0eabbd4391ef93ccd49d0f2747f4f6a2a2738e99e5adc65/requests-2.26.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7e/e2/0c8be1e3b237d11f10963c7cef82e4d98d70ca6e176f3922dddf7bfc7026/python_slugify-5.0.2-py2.py3-none-any.whl
  Using ca

In [3]:
from pyspark.sql import SparkSession, SQLContext,DataFrame
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import when,lit,isnan,udf, count, col,isnull,array,abs,sqrt,pow,hour,month,dayofweek, to_date, avg
import pyspark.sql.functions as func
import time
from pyspark.sql.types import FloatType, IntegerType
import plotly.graph_objects as go
import plotly
import matplotlib.pyplot as plt
import copy 
import math
import numpy as np
from sklearn.metrics import classification_report
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler, VectorAssembler

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
import os
os.environ["KAGGLE_USERNAME"]="guyassa"
os.environ["KAGGLE_KEY"]="01f262ffbee65b00b91ca894e7a3f964"
os.chdir('/tmp')
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
api.competition_download_files('idc-big-data-platforms-2021-ml-competition')
import zipfile
with zipfile.ZipFile('./idc-big-data-platforms-2021-ml-competition.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def getFileSystem(sc):
    # Prepare a FileSystem manager
    FileSystem = sc._jvm.org.apache.hadoop.fs.FileSystem
    fs = FileSystem.get(sc._jsc.hadoopConfiguration())
    return fs

def copy_from_local_file(sc, logger, local_file, hdfs_file, delSrc=False, overwrite=True):
    # copyFromLocalFile(boolean delSrc, boolean overwrite, Path src, Path dst)
    Path = sc._jvm.org.apache.hadoop.fs.Path
    try:
        getFileSystem(sc).copyFromLocalFile(delSrc, overwrite, Path(local_file), Path(hdfs_file))
        logger.info("copyFromLocal {} to {} success".format(local_file, hdfs_file))
    except Exception as e:
        logger.error(e)
        logger.info("copyFromLocal {} to {} error".format(local_file, hdfs_file))
import logging
import sys
logger = logging.getLogger('')
fs = getFileSystem(sc)
copy_from_local_file(sc, logger, '/tmp/idc_train.csv', '/user/livy/idc_train.csv')
copy_from_local_file(sc, logger, '/tmp/idc_test.csv', '/user/livy/idc_test.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# load train data
train = spark.read.format("csv") \
      .option("header", True) \
      .option("inferSchema", True) \
      .load("idc_train.csv")
train.cache() 
train.printSchema()
train.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- key: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- passenger_count: integer (nullable = true)

19990000

# Settings

In [7]:
t = time.time()
train = spark.read.csv("idc_train.csv",header=True,)
print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Run time:  0.2212233543395996 seconds

In [8]:
# adding "high_fare" column
t = time.time()

train = train.withColumn("high_fare",when((train.fare_amount>=10),1).otherwise(0))

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Run time:  0.07002043724060059 seconds

In [9]:
# sanity check
t = time.time()
train.groupby('high_fare').count().show()
print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------+
|high_fare|   count|
+---------+--------+
|        1| 7789236|
|        0|12200764|
+---------+--------+

Run time:  3.362328290939331 seconds

In [10]:
# passenger - from string to integer
# Statistics
t = time.time()

train = train.withColumn("fare_amount", train["fare_amount"].cast(IntegerType()))
train = train.withColumn("passenger_count", train["passenger_count"].cast(IntegerType()))
train = train.withColumn('pickup_longitude', train['pickup_longitude'].cast('float'))
train = train.withColumn('pickup_latitude', train['pickup_latitude'].cast('float'))
train = train.withColumn('dropoff_longitude', train['dropoff_longitude'].cast('float'))
train = train.withColumn('dropoff_latitude', train['dropoff_latitude'].cast('float'))

numeric_features = [t[0] for t in train.dtypes if t[1] == 'int']
train.select(numeric_features).describe().toPandas().transpose()

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Run time:  3.332919120788574 seconds

In [11]:
# Set unwanted values as Null (such as fare_amount=0 etc.)
t = time.time()

train = train.withColumn("fare_amount",when(train.fare_amount==0,np.nan).otherwise(train.fare_amount))
train = train.withColumn("fare_amount",when(train.fare_amount>100,np.nan).otherwise(train.fare_amount))
train = train.withColumn("fare_amount",when(train.fare_amount<0,np.nan).otherwise(train.fare_amount))
train = train.withColumn("pickup_datetime",when(train.pickup_datetime==0,np.nan).otherwise(train.pickup_datetime))
train = train.withColumn("pickup_longitude",when(train.pickup_longitude==0,np.nan).otherwise(train.pickup_longitude))
train = train.withColumn("pickup_latitude",when(train.pickup_latitude==0,np.nan).otherwise(train.pickup_latitude))
train = train.withColumn("dropoff_longitude",when(train.dropoff_longitude==0,np.nan).otherwise(train.dropoff_longitude))
train = train.withColumn("dropoff_latitude",when(train.dropoff_latitude==0,np.nan).otherwise(train.dropoff_latitude))
train = train.withColumn("passenger_count",when(train.passenger_count==0,np.nan).otherwise(train.passenger_count))
train = train.withColumn("passenger_count",when(train.passenger_count>5,np.nan).otherwise(train.passenger_count))

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Run time:  0.1959683895111084 seconds

In [12]:
# Observe the change in samples and look if imbalance occur
t = time.time()

train = train.na.drop()
train.groupby('high_fare').count().show()

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------+
|high_fare|   count|
+---------+--------+
|        1| 7425626|
|        0|11673191|
+---------+--------+

Run time:  6.9476072788238525 seconds

## Feature engineering

In [13]:
# convert wgs84 coordinate (as given in data) to distance

def distance(lat1, lon1, lat2, lon2):
    
    radius = 6371  # km

    dlat = func.radians(lat2 - lat1)
    dlon = func.radians(lon2 - lon1)
    a = (func.sin(dlat / 2) * func.sin(dlat / 2) +
         func.cos(func.radians(lat1)) * func.cos(func.radians(lat2)) *
         func.sin(dlon / 2) * func.sin(dlon / 2))
    c = 2 * func.atan2(func.sqrt(a), func.sqrt(1 - a))
    d = radius * c

    return d

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# adding relevanr columns
t = time.time()

# adding hour_of_day column 
train = train.withColumn("hour_of_day", hour(col('pickup_datetime')))

# adding day_of_week column
train = train.withColumn("day_of_week", when((train.fare_amount>0),lit(dayofweek(train.pickup_datetime))).otherwise(lit(0)))

# adding month_of_year column 
train = train.withColumn("month_of_year",when((train.fare_amount>0),lit(month(train.pickup_datetime))).otherwise(lit(0)))

# add column of distance as calculated in the function
train = train.withColumn("distance", distance(col("pickup_latitude"), col("pickup_longitude"), col("dropoff_latitude"), col("dropoff_longitude")))

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Run time:  0.13969659805297852 seconds

# Plotting

In [17]:
fraction = 10_000 / train.count()
#Randomly sample fraction% of the data without replacement
sample1 = train.sample(False, fraction, seed=0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
sample1.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----------+--------------------+------------------+------------------+------------------+------------------+---------------+---------+-----------+-----------+-------------+-------------------+
|  key|fare_amount|     pickup_datetime|  pickup_longitude|   pickup_latitude| dropoff_longitude|  dropoff_latitude|passenger_count|high_fare|hour_of_day|day_of_week|month_of_year|           distance|
+-----+-----------+--------------------+------------------+------------------+------------------+------------------+---------------+---------+-----------+-----------+-------------+-------------------+
| 1779|       13.0|2013-06-01 21:30:...| -73.9852294921875|  40.7679557800293|-73.99337768554688| 40.73308181762695|            1.0|        1|         21|          7|            6|  3.938084052506227|
| 4203|       34.0|2011-10-06 14:02:...|-73.99268341064453| 40.76862716674805|-73.87047576904297| 40.77334213256836|            1.0|        1|         14|          5|           10|  10.30454756553

In [26]:
# 7.a- not normalized
t = time.time()
import plotly.express as px
df = sample1.toPandas()
fig = px.histogram(df, x="fare_amount")
fig.update_layout(title="Distribution of ride fairs")
fig.show()
print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# 7.a normalized
t = time.time()
import plotly.graph_objects as go

import numpy as np

x = sample1.toPandas()
x=x["fare_amount"]
fig = go.Figure(data=[go.Histogram(x=x, histnorm='probability')])

fig.update_layout(
    title="Distribution of ride fairs")

fig.show()
print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Run time:  14.994751691818237 seconds

In [28]:
# 7b- version A
t = time.time()
import plotly.express as px
data_ = sample1.toPandas()
fig = px.bar(data_, x='fare_amount', y='distance')
fig.update_layout(
    title="Ride fare vs. ride length")

fig.show()
print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Run time:  14.954370975494385 seconds

In [29]:
# 7b- version B
t = time.time()
import plotly.express as px
df = sample1.toPandas()

# 2 options 
fig = px.scatter(df,x="fare_amount", y="distance")
# fig = px.scatter(df,x="distance", y="fare_amount")
fig.update_layout(
    title="Ride fare vs. ride length")
fig.show()
print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Run time:  15.031952857971191 seconds

In [30]:
# 7.c - version A
t = time.time()
import plotly.express as px
df =  sample1.toPandas()
fig = px.histogram(df, x="hour_of_day", y="distance", histfunc='avg')

fig.update_layout(bargap=0.30,bargroupgap=0.0,title="Average ride length per pick up time hour of the day (x-axis = 24 hours)")

fig.show()
print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Run time:  15.532449960708618 seconds

# More preparetions

In [31]:
# select the relevant columns and making tuple column from all relevant columns (features)

t = time.time()

selected = ["high_fare", "passenger_count", "distance", 
            "hour_of_day", "day_of_week"]

filtered_train = train.select(*selected)

# making column as a tuple of all features
vectored = VectorAssembler(inputCols=['passenger_count','distance','hour_of_day','day_of_week'], outputCol='vector')
vectored.setHandleInvalid("skip").transform(filtered_train)
filtered_train = vectored.transform(filtered_train)


filtered_train.show()

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------------+-------------------+-----------+-----------+--------------------+
|high_fare|passenger_count|           distance|hour_of_day|day_of_week|              vector|
+---------+---------------+-------------------+-----------+-----------+--------------------+
|        0|            1.0| 2.8847879409444266|         19|          3|[1.0,2.8847879409...|
|        1|            1.0|  3.745045383430712|         20|          3|[1.0,3.7450453834...|
|        0|            1.0|  1.304617552457649|         13|          1|[1.0,1.3046175524...|
|        0|            1.0| 2.5238617905217566|         19|          7|[1.0,2.5238617905...|
|        0|            1.0| 1.9369673352472951|         13|          7|[1.0,1.9369673352...|
|        1|            2.0| 20.748283643692666|         14|          6|[2.0,20.748283643...|
|        0|            1.0|  3.198885565601612|         22|          1|[1.0,3.1988855656...|
|        0|            1.0| 2.0023518724222575|         15|          4

In [32]:
# StandardScaler - scaling the data
t = time.time()
scalar = StandardScaler().setInputCol('vector').setOutputCol('scaled_vector')
scalar_model = scalar.fit(filtered_train)
scalar_train = scalar_model.transform(filtered_train)

scalar_train.show()
print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------------+-------------------+-----------+-----------+--------------------+--------------------+
|high_fare|passenger_count|           distance|hour_of_day|day_of_week|              vector|       scaled_vector|
+---------+---------------+-------------------+-----------+-----------+--------------------+--------------------+
|        0|            1.0| 2.8847879409444266|         19|          3|[1.0,2.8847879409...|[0.86542381091983...|
|        1|            1.0|  3.745045383430712|         20|          3|[1.0,3.7450453834...|[0.86542381091983...|
|        0|            1.0|  1.304617552457649|         13|          1|[1.0,1.3046175524...|[0.86542381091983...|
|        0|            1.0| 2.5238617905217566|         19|          7|[1.0,2.5238617905...|[0.86542381091983...|
|        0|            1.0| 1.9369673352472951|         13|          7|[1.0,1.9369673352...|[0.86542381091983...|
|        1|            2.0| 20.748283643692666|         14|          6|[2.0,20.748283643

In [33]:
#split to train and validation sets

t = time.time()

train_set, val_set = scalar_train.randomSplit([0.8, 0.2], seed=12345)

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Run time:  0.019936323165893555 seconds

In [34]:
# Let us check whether their is imbalance in the dataset
dataset_size=float(train_set.select("high_fare").count())
numPositives=train_set.select("high_fare").where('high_fare == 1').count()
per_ones=(float(numPositives)/float(dataset_size))*100
numNegatives=float(dataset_size-numPositives)
print('The number of ones are {}'.format(numPositives))
print('Percentage of ones are {}'.format(per_ones))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The number of ones are 5941498
Percentage of ones are 38.882128768963184

In [35]:
# Imbalancing handling - by adding weights
t = time.time()

BalancingRatio= numNegatives/dataset_size
print('BalancingRatio = {}'.format(BalancingRatio))

# creating a new column named “classWeights” in the “train” dataset
train_set = train_set.withColumn("classWeights", when(train_set.high_fare == 1,BalancingRatio).otherwise(1-BalancingRatio))

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

BalancingRatio = 0.6111787123103681
Run time:  0.02907729148864746 seconds

# Models

In [36]:
# Create the evaluator for all models for AUC calculations

from pyspark.ml.evaluation import BinaryClassificationEvaluator

t = time.time()
evaluator = BinaryClassificationEvaluator(labelCol='high_fare',metricName='areaUnderROC')

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Run time:  0.018456459045410156 seconds

### Logistic Regression

In [37]:
# train logistic regression
from pyspark.ml.classification import LogisticRegression

t = time.time()

lr = LogisticRegression(labelCol="high_fare", featuresCol="scaled_vector", weightCol="classWeights")
lr_model = lr.fit(train_set)
# p_train_lr = lr_model.transform(train_set)
predictions_lr = lr_model.transform(val_set)

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Run time:  140.89545392990112 seconds

#### test on validation

In [38]:
# confusion matrix:
t = time.time()

from sklearn.metrics import classification_report, confusion_matrix

y_true_lr = predictions_lr.select(['high_fare']).collect()
y_pred_lr = predictions_lr.select(['prediction']).collect()

print('The LR confusion matrix:')
print(confusion_matrix(y_true_lr, y_pred_lr))

print(classification_report(y_true_lr, y_pred_lr))


print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The LR confusion matrix:
[[2257766   76129]
 [ 501952  982176]]
              precision    recall  f1-score   support

           0       0.82      0.97      0.89   2333895
           1       0.93      0.66      0.77   1484128

    accuracy                           0.85   3818023
   macro avg       0.87      0.81      0.83   3818023
weighted avg       0.86      0.85      0.84   3818023

Run time:  403.32832384109497 seconds

In [39]:
# AUC

# Apply evaluator to data 

t = time.time()
# evaluation_train = evaluator.evaluate(p_train_lr)
evaluation_test_lr = evaluator.evaluate(predictions_lr)

# The result of evaluate is, by default, the "Area Under Curve":
# print("evaluation train (area under ROC): %f" % evaluation_train)
print("evaluation test (area under ROC): %f" % evaluation_test_lr)

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluation test (area under ROC): 0.917304
Run time:  69.01463842391968 seconds

### Decision Tree

In [40]:
# train decision tree
from pyspark.ml.classification import DecisionTreeClassifier

t = time.time()

dt = DecisionTreeClassifier(labelCol="high_fare", featuresCol="scaled_vector", weightCol="classWeights")
dt_model = dt.fit(train_set)
# p_train_dt = dt_model.transform(train_set)
predictions_dt = dt_model.transform(val_set)

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Run time:  299.4897127151489 seconds

#### test on validation

In [41]:
# confusion matrix

t = time.time()

from sklearn.metrics import classification_report, confusion_matrix

y_true_dt = predictions_dt.select(['high_fare']).collect()
y_pred_dt = predictions_dt.select(['prediction']).collect()

print('The decision tree confusion matrix:')
print(confusion_matrix(y_true_dt, y_pred_dt))

print(classification_report(y_true_dt, y_pred_dt))

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The decision tree confusion matrix:
[[2001786  332109]
 [ 221836 1262292]]
              precision    recall  f1-score   support

           0       0.90      0.86      0.88   2333895
           1       0.79      0.85      0.82   1484128

    accuracy                           0.85   3818023
   macro avg       0.85      0.85      0.85   3818023
weighted avg       0.86      0.85      0.86   3818023

Run time:  402.47747588157654 seconds

In [42]:
# AUC

# Apply evaluator to data 

t = time.time()
# evaluation_train = evaluator.evaluate(p_train_lr)
evaluation_test_dt = evaluator.evaluate(predictions_dt)

# The result of evaluate is, by default, the "Area Under Curve":
# print("evaluation train (area under ROC): %f" % evaluation_train)
print("evaluation test (area under ROC): %f" % evaluation_test_dt)

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluation test (area under ROC): 0.744871
Run time:  65.88747978210449 seconds

### Gradient-boosted Tree

In [43]:
# train gradient-boosted tree

from pyspark.ml.classification import GBTClassifier

t = time.time()

gbt = GBTClassifier(labelCol="high_fare", featuresCol="scaled_vector", weightCol="classWeights")
gbt_model = gbt.fit(train_set)
# p_train_gbt = gbt_model.transform(train_set)
predictions_gbt = gbt_model.transform(val_set)

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Run time:  308.04555463790894 seconds

#### test on validation

In [44]:
# confusion matrix

t = time.time()

from sklearn.metrics import classification_report, confusion_matrix

y_true_gbt = predictions_gbt.select(['high_fare']).collect()
y_pred_gbt = predictions_gbt.select(['prediction']).collect()

print('The GBT confusion matrix:')
print(confusion_matrix(y_true_gbt, y_pred_gbt))

print(classification_report(y_true_gbt, y_pred_gbt))

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The GBT confusion matrix:
[[2055682  278213]
 [ 245805 1238323]]
              precision    recall  f1-score   support

           0       0.89      0.88      0.89   2333895
           1       0.82      0.83      0.83   1484128

    accuracy                           0.86   3818023
   macro avg       0.85      0.86      0.86   3818023
weighted avg       0.86      0.86      0.86   3818023

Run time:  403.7405741214752 seconds

In [45]:
# AUC

# Apply evaluator to data 

t = time.time()
# evaluation_train = evaluator.evaluate(p_train_lr)
evaluation_test_gbt = evaluator.evaluate(predictions_gbt)

# The result of evaluate is, by default, the "Area Under Curve":
# print("evaluation train (area under ROC): %f" % evaluation_train)
print("evaluation test (area under ROC): %f" % evaluation_test_gbt)

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluation test (area under ROC): 0.931808
Run time:  64.75766110420227 seconds

### Random Forest

In [46]:
from pyspark.ml.classification import RandomForestClassifier

t = time.time()

rf = RandomForestClassifier(labelCol="high_fare", featuresCol="scaled_vector", weightCol="classWeights",numTrees=20)
rf_model = rf.fit(train_set)
# p_train_dt = rf_model.transform(train_set)
predictions_rf = rf_model.transform(val_set)

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Run time:  323.69037795066833 seconds

#### test on validation

In [47]:
# confusion matrix

t = time.time()

from sklearn.metrics import classification_report, confusion_matrix

y_true_rf = predictions_rf.select(['high_fare']).collect()
y_pred_rf = predictions_rf.select(['prediction']).collect()

print('The RF confusion matrix:')
print(confusion_matrix(y_true_rf, y_pred_rf))

print(classification_report(y_true_rf, y_pred_rf))

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The RF confusion matrix:
[[2072443  261452]
 [ 268579 1215549]]
              precision    recall  f1-score   support

           0       0.89      0.89      0.89   2333895
           1       0.82      0.82      0.82   1484128

    accuracy                           0.86   3818023
   macro avg       0.85      0.85      0.85   3818023
weighted avg       0.86      0.86      0.86   3818023

Run time:  406.61516070365906 seconds

In [48]:
# AUC

# Apply evaluator to data 

t = time.time()
# evaluation_train = evaluator.evaluate(p_train_lr)
evaluation_test_rf = evaluator.evaluate(predictions_rf)

# The result of evaluate is, by default, the "Area Under Curve":
# print("evaluation train (area under ROC): %f" % evaluation_train)
print("evaluation test (area under ROC): %f" % evaluation_test_rf)

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluation test (area under ROC): 0.903696
Run time:  65.05274105072021 seconds

### Best result

In [49]:
# Best result based on val set:
t = time.time()
d = {'LR': evaluation_test_lr, 'Decision Tree': evaluation_test_dt, 'GBT': evaluation_test_gbt, 'RF': evaluation_test_rf}
dd = {'LR': lr_model, 'Decision Tree': dt_model, 'GBT': gbt_model, 'RF': rf_model}
chosen = max(d, key=d.get)
chosen_model = dd[chosen]
print('The best algorithm is: ', chosen)
print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The best algorithm is:  GBT
Run time:  0.00019979476928710938 seconds

# Test the data set

### preparetions

In [50]:
# load test
t = time.time()
df_test = spark.read.format("csv") \
      .option("header", True) \
      .option("inferSchema", True) \
      .load("idc_test.csv")

df_test.printSchema()
print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- key: integer (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- passenger_count: integer (nullable = true)

Run time:  0.207688570022583 seconds

In [51]:
# Set unwanted values as Null (such as fare_amount=0 etc.)
t = time.time()

df_test = df_test.withColumn("pickup_datetime",when(df_test.pickup_datetime==0,np.nan).otherwise(df_test.pickup_datetime))
df_test = df_test.withColumn("pickup_longitude",when(df_test.pickup_longitude==0,np.nan).otherwise(df_test.pickup_longitude))
df_test = df_test.withColumn("pickup_latitude",when(df_test.pickup_latitude==0,np.nan).otherwise(df_test.pickup_latitude))
df_test = df_test.withColumn("dropoff_longitude",when(df_test.dropoff_longitude==0,np.nan).otherwise(df_test.dropoff_longitude))
df_test = df_test.withColumn("dropoff_latitude",when(df_test.dropoff_latitude==0,np.nan).otherwise(df_test.dropoff_latitude))
df_test = df_test.withColumn("passenger_count",when(df_test.passenger_count==0,np.nan).otherwise(df_test.passenger_count))
df_test = df_test.withColumn("passenger_count",when(df_test.passenger_count>5,np.nan).otherwise(df_test.passenger_count))

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Run time:  0.07856297492980957 seconds

In [52]:
# passenger - from string to integer
# Statistics
t = time.time()

df_test = df_test.withColumn("passenger_count", df_test["passenger_count"].cast(IntegerType()))
df_test = df_test.withColumn('pickup_longitude', df_test['pickup_longitude'].cast('float'))
df_test = df_test.withColumn('pickup_latitude', df_test['pickup_latitude'].cast('float'))
df_test = df_test.withColumn('dropoff_longitude', df_test['dropoff_longitude'].cast('float'))
df_test = df_test.withColumn('dropoff_latitude', df_test['dropoff_latitude'].cast('float'))

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Run time:  0.026969432830810547 seconds

In [53]:
from pyspark.ml.feature import Imputer
imputer=Imputer(inputCols=["pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude","passenger_count"],
                outputCols=["pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude","passenger_count"])

model=imputer.fit(df_test)
df_test = model.transform(df_test)

df_test.show()
# imputer.setStrategy("median").setMissingValue(1.0).fit(df_test).transform(df_test).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+----------------+---------------+-----------------+----------------+---------------+
|     key|     pickup_datetime|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|passenger_count|
+--------+--------------------+----------------+---------------+-----------------+----------------+---------------+
|19990001|2012-05-31 07:04:...|       -73.94636|      40.792088|        -73.96618|       40.764984|              2|
|19990002|2015-01-27 20:51:...|       -73.97137|        40.7604|       -73.995346|       40.744392|              0|
|19990003|2015-05-18 16:41:...|       -73.96013|       40.76694|        -73.95677|        40.77497|              5|
|19990004|2009-05-30 04:13:...|       -73.98996|       40.72943|       -73.994835|       40.743565|              1|
|19990005|2010-04-08 07:55:...|       -73.96511|      40.791817|        -73.94468|       40.773884|              1|
|19990006|2010-03-05 21:35:...|       -73.98098|       40.75984|        

### Feature engineering

In [54]:
# adding relevanr columns
t = time.time()

# adding hour_of_day column 
df_test = df_test.withColumn("hour_of_day", hour(col('pickup_datetime')))

# adding day_of_week column
df_test = df_test.withColumn("day_of_week", dayofweek(col('pickup_datetime')))

# adding month_of_year column 
df_test = df_test.withColumn("month_of_year", month(col('pickup_datetime')))

# add column of distance as calculated in the function
df_test = df_test.withColumn("distance", distance(col("pickup_latitude"), col("pickup_longitude"), col("dropoff_latitude"), col("dropoff_longitude")))

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Run time:  0.10367178916931152 seconds

In [55]:
# select the relevant columns and making tuple column from all relevant columns (features)

t = time.time()

selected = ["key", "passenger_count", "distance", 
            "hour_of_day", "day_of_week"]

filtered_test = df_test.select(*selected)

# making column as a tuple of all features
vectored = VectorAssembler(inputCols=['passenger_count','distance','hour_of_day','day_of_week'], outputCol='vector')
vectored.setHandleInvalid("skip").transform(filtered_test)
filtered_test = vectored.transform(filtered_test)


filtered_test.show()

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------------+------------------+-----------+-----------+--------------------+
|     key|passenger_count|          distance|hour_of_day|day_of_week|              vector|
+--------+---------------+------------------+-----------+-----------+--------------------+
|19990001|              2| 3.445027998950589|          7|          5|[2.0,3.4450279989...|
|19990002|              0| 2.692155938508404|         20|          3|[0.0,2.6921559385...|
|19990003|              5|0.9365699721459226|         16|          2|[5.0,0.9365699721...|
|19990004|              1|1.6243606370807382|          4|          7|[1.0,1.6243606370...|
|19990005|              1| 2.633528396391398|          7|          5|[1.0,2.6335283963...|
|19990006|              1|1.8972364234079255|         21|          6|[1.0,1.8972364234...|
|19990007|              1|1.9831838558644683|         14|          6|[1.0,1.9831838558...|
|19990008|              1| 1.687381284474482|         16|          2|[1.0,1.6873812844...|

In [56]:
# StandardScaler - scaling the data
t = time.time()
scalar = StandardScaler().setInputCol('vector').setOutputCol('scaled_vector')
scalar_model = scalar.fit(filtered_test)
scalar_test = scalar_model.transform(filtered_test)

scalar_test.show()
print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------------+------------------+-----------+-----------+--------------------+--------------------+
|     key|passenger_count|          distance|hour_of_day|day_of_week|              vector|       scaled_vector|
+--------+---------------+------------------+-----------+-----------+--------------------+--------------------+
|19990001|              2| 3.445027998950589|          7|          5|[2.0,3.4450279989...|[1.70746675972262...|
|19990002|              0| 2.692155938508404|         20|          3|[0.0,2.6921559385...|[0.0,0.0275662682...|
|19990003|              5|0.9365699721459226|         16|          2|[5.0,0.9365699721...|[4.26866689930655...|
|19990004|              1|1.6243606370807382|          4|          7|[1.0,1.6243606370...|[0.85373337986131...|
|19990005|              1| 2.633528396391398|          7|          5|[1.0,2.6335283963...|[0.85373337986131...|
|19990006|              1|1.8972364234079255|         21|          6|[1.0,1.8972364234...|[0.85373337986

### Prediction

In [57]:
# predict with best fit:
t = time.time()

pretict_test = chosen_model.transform(scalar_test) #### Choose automatically from the best score model
pretict_test.groupBy('prediction').count().show()


print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|prediction|count|
+----------+-----+
|       1.0| 3414|
|       0.0| 6586|
+----------+-----+

Run time:  1.2532753944396973 seconds

In [64]:
# final result:
t = time.time()

final_test = pretict_test.select('key', 'prediction')
final_test = final_test.withColumnRenamed("prediction", "high_fare")
final_test.show()

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+
|     key|high_fare|
+--------+---------+
|19990001|      1.0|
|19990002|      0.0|
|19990003|      0.0|
|19990004|      0.0|
|19990005|      0.0|
|19990006|      0.0|
|19990007|      0.0|
|19990008|      0.0|
|19990009|      0.0|
|19990010|      0.0|
|19990011|      1.0|
|19990012|      1.0|
|19990013|      0.0|
|19990014|      0.0|
|19990015|      1.0|
|19990016|      0.0|
|19990017|      0.0|
|19990018|      0.0|
|19990019|      1.0|
|19990020|      1.0|
+--------+---------+
only showing top 20 rows

Run time:  0.3719048500061035 seconds

In [65]:
final_test = final_test.withColumnRenamed("prediction", "high_fare")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
t = time.time()
final_test.write.option("header", "true").csv('prediction_file3.csv')

print('Run time: ', time.time() - t , 'seconds')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Run time:  1.0849847793579102 seconds

In [53]:
# SparkSession.stop(spark)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:

    
    
path = 's3://aws-emr-resources-432905257568-us-east-1/notebooks/prediction_file3.csv'

final_test.repartition(1) \
.write \
.mode ('overwrite') \
.option('header', 'true') \
.option('sep',',') \
.csv(path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…